In [1]:
import numpy as np
from function import RBF_kernel,linear_kernel
from sklearn.metrics.pairwise import laplacian_kernel
import scipy
from scipy.stats import multivariate_normal
from scipy.stats._multivariate import _PSD
from scipy.stats._multivariate import multivariate_normal_gen
#np.random.seed(1234)

In [2]:
import os
# path = '/Users/lilingyu/E/PhD/Python/'
# path = '/storage3/yhhuang/users/lingyu/'
# path = '/data/users/lingyu/'
# path = '/home/lly/Python/'
#path = '/mnt/lingyu/nfs_share2/Python/'
path = '/mnt/jackywang/nfs_share2/spatialde'

## Function

In [3]:
def _pinv_1d(v, eps=1e-5):
    return np.array([0 if abs(x) <= eps else 1/x for x in v], dtype=float)

In [ ]:
# class new_normal(multivariate_normal_gen):
#     def __init__(self,seed = None):
#         super().__init__(seed)
    
#     def 

In [4]:
class Gaussian:

    def __init__(self,mean,cov,spatial,ss_loc,dependency=None,d=5,kernel='RBF',l=0.01):
        """
        number of superspots: M
        mean: N-length 1-d array
        cov: full pre-determined covariance matrix
        dependency format: M-length list of lists, the i-th element indicates the superspots that the i-th superspot is dependent on
        ss_loc: M-length list of lists, the i-th element indicates the spots (ordinal) that the i-th superspots contain
        l: hyperparameter for kernel
        """
        super().__init__()
        if mean.ndim == 1:
            self.all_mean = np.array(mean)[:,np.newaxis]
        else:
            self.all_mean = np.array(mean)       
        self.all_cov = cov
        self.dependency = dependency   # temporarily assuming dependency[i] > i 
        self.M = len(dependency)
        self.ss_loc = ss_loc
        self.spatial = spatial
        self.cond_mean = None
        self.l = l
        self.d = d
        self._initialize()
    
    def _initialize(self):
        self._init_ds_loc()
        self._init_inv()
        self._init_ss_cov()        
        #self._init_cond_cov()

    def _init_ds_loc(self):
        self.ds_loc = []        
        for i,ds in enumerate(self.dependency):
           ind = []
           for ss in ds:
               ind += self.ss_loc[ss]
           self.ds_loc.append(ind)
    
    def _init_inv(self):
        self.ds_inv_sums = []
        self.ds_inv_col_sums = []
        for i,loc in enumerate(self.ds_loc):
            ds_cov = self.all_cov[np.ix_(loc,loc)]
            if len(ds_cov) == 0:
                self.ds_inv_col_sums.append(ds_cov)
                self.ds_inv_sums.append(ds_cov)
            else:
                inv = np.linalg.solve(ds_cov, np.identity(len(ds_cov)))
                self.ds_inv_col_sums.append(np.sum(inv,axis=0))
                self.ds_inv_sums.append(np.sum(inv))

    def _init_ss_cov(self):       #M*M covariance between superspots
        mean_loc = []
        for i,loc in enumerate(self.ss_loc):
            mean_loc.append(np.average(self.spatial[loc],axis=0))
        mean_loc = np.array(mean_loc)
        #self.ss_cov = RBF_kernel(mean_loc,self.l)
        self.ss_cov = laplacian_kernel(mean_loc)
        print(self.ss_cov)

    
    # def _init_cond_cov(self):
    #     eps = 1e-5
    #     self.cond_U = []
    #     self.log_pdet = []
    #     for i in range(self.M):
    #         in_cov = self.all_cov[np.ix_(self.ss_loc[i],self.ss_loc[i])]   
    #         if len(self.dependency[i]) == 0:
    #             cond_cov = in_cov
    #         else:
    #             cond_cov = in_cov - np.ones_like(in_cov)*self.ds_inv_sums[i]*np.average(self.ss_cov[i,self.dependency[i]])**2
    #         #treat cond_cov as positive definite 
    #         s, u = scipy.linalg.eigh(cond_cov)
    #         #print(len(s[s<=0]))
    #         d = s[s > eps]
    #         s_pinv = _pinv_1d(s,eps)
    #         print(len(s_pinv[s_pinv<eps]),len(s_pinv[s_pinv<-eps]))
    #         print(s_pinv[s_pinv<-eps])
    #         self.cond_U.append(np.multiply(u, np.sqrt(s_pinv)))
    #         #print(np.isnan(self.cond_U).any())
    #         self.log_pdet.append(np.sum(np.log(d)))
    
        
 
    def get_cond_dev(self,dev, i=0, y=None, all_mean=None):    #dev:N*G
        return dev[self.ss_loc[i]] -  np.average(self.ss_cov[i,self.dependency[i]])*self.ds_inv_col_sums[i] @ dev[self.ds_loc[i]]

     
    # def gs_nll_total(self,Y):
    #     """
    #     Y: observation of dimension N*G
    #     """
    #     N,G = Y.shape
    #     ll = np.log(2 * np.pi)*N*G + np.sum(self.log_pdet)*G
    #     dev = Y - self.all_mean    #N*G
    #     for i in range(self.M):
    #         U = self.cond_U[i]
    #         if len(self.dependency[i]) == 0:
    #             cond_dev = dev[self.ss_loc[i]]
    #         else:
    #             cond_dev = self.get_cond_dev(dev,i)
    #         for g in range(G):
    #             ll += np.sum(np.square(cond_dev[:,g].T @ U), axis=-1)
    #     return -0.5*ll
    
    # def gs_nll_sep(self,Y):    
    #     """
    #     Y: observation of dimension N*G
    #     """
    #     N,G = Y.shape
    #     ll = np.log(2 * np.pi)*N + np.sum(self.log_pdet)
    #     result = [-0.5*ll]*G
    #     print(result)
    #     dev = Y - self.all_mean    #N*G
    #     for i in range(self.M):
    #         U = self.cond_U[i]
    #         if len(self.dependency[i]) == 0:
    #             cond_dev = dev[self.ss_loc[i]]
    #         else:
    #             cond_dev = self.get_cond_dev(dev,i)
    #         for g in range(G):
    #             result[g] += -0.5*np.sum(np.square(cond_dev[:,g].T @ U), axis=-1)
    #     return np.array(result)
    


In [28]:
class Gaussian_psd(Gaussian):
    def __init__(self,mean,cov,spatial,ss_loc,dependency=None,d=5,kernel='RBF',l=0.01):
        super().__init__(mean,cov,spatial,ss_loc,dependency,d,kernel,l)
        self._init_cond_cov()
        
        
    def _init_cond_cov(self):
        eps = 1e-5
        self.cond_U = []
        self.log_pdet = []
        for i in range(self.M):
            in_cov = self.all_cov[np.ix_(self.ss_loc[i],self.ss_loc[i])]   
            if len(self.dependency[i]) == 0:
                cond_cov = in_cov
            else:
                cond_cov = in_cov - np.ones_like(in_cov)*self.ds_inv_sums[i]*np.average(self.ss_cov[i,self.dependency[i]])**2
            #treat cond_cov as positive definite 
            s, u = scipy.linalg.eigh(cond_cov)
            #print(len(s[s<=0]))
            d = s[s > eps]
            s_pinv = _pinv_1d(s,eps)
            print(len(s_pinv[s_pinv<eps]),len(s_pinv[s_pinv<-eps]))
            print(s_pinv[s_pinv<-eps])
            self.cond_U.append(np.multiply(u, np.sqrt(s_pinv)))
            #print(np.isnan(self.cond_U).any())
            self.log_pdet.append(np.sum(np.log(d)))
    
    def gs_nll_sep(self,Y):    
        """
        Y: observation of dimension N*G
        """
        N,G = Y.shape
        ll = np.log(2 * np.pi)*N + np.sum(self.log_pdet)
        result = [-0.5*ll]*G
        #print(result)
        dev = Y - self.all_mean    #N*G
        for i in range(self.M):
            U = self.cond_U[i]
            if len(self.dependency[i]) == 0:
                cond_dev = dev[self.ss_loc[i]]
            else:
                cond_dev = self.get_cond_dev(dev,i)
            for g in range(G):
                result[g] += -0.5*np.sum(np.square(cond_dev[:,g].T @ U), axis=-1)
        return np.array(result)
    

    
class Gaussian_gen(Gaussian):
    def __init__(self,mean,cov,spatial,ss_loc,dependency=None,d=5,kernel='RBF',l=0.01):
        super().__init__(mean,cov,spatial,ss_loc,dependency,d,kernel,l)
        self._init_cond_cov()
        
        
    def _init_cond_cov(self):
        eps = 1e-5
        self.cond_inv = []
        self.log_pdet = []
        for i in range(self.M):
            in_cov = self.all_cov[np.ix_(self.ss_loc[i],self.ss_loc[i])]   
            if len(self.dependency[i]) == 0:
                cond_cov = in_cov
            else:
                cond_cov = in_cov - np.ones_like(in_cov)*self.ds_inv_sums[i]*np.average(self.ss_cov[i,self.dependency[i]])**2
            #treat cond_cov as positive definite 
            self.cond_inv.append(np.linalg.inv(cond_cov))
            #print(np.linalg.det(cond_cov))
            #self.log_pdet.append(np.log(np.linalg.det(cond_cov)))
            s, u = scipy.linalg.eigh(cond_cov)
            #print(len(s[s<=0]))
            d = s[s > eps]
            s_pinv = _pinv_1d(s,eps)
            # print(len(s_pinv[s_pinv<eps]),len(s_pinv[s_pinv<-eps]))
            print(s_pinv[s_pinv<eps])
            print("det:",np.sum(np.log(d)))
            # self.cond_U.append(np.multiply(u, np.sqrt(s_pinv)))
            # #print(np.isnan(self.cond_U).any())
            # self.log_pdet.append(np.sum(np.log(d)))
    
    def gs_nll_sep(self,Y):    
        """
        Y: observation of dimension N*G
        """
        N,G = Y.shape
        ll = np.log(2 * np.pi)*N + np.sum(self.log_pdet)
        result = [-0.5*ll]*G
        dev = Y - self.all_mean    #N*G
        for i in range(self.M):
            inv = self.cond_inv[i]
            if len(self.dependency[i]) == 0:
                cond_dev = dev[self.ss_loc[i]]
            else:
                cond_dev = self.get_cond_dev(dev,i)
            for g in range(G):
                result[g] += -0.5* (cond_dev[:,g].T @ inv @ cond_dev[:,g])
        return np.array(result)

In [6]:
def get_l_limits(X):
    Xsq = np.sum(np.square(X), 1)
    R2 = -2. * np.dot(X, X.T) + (Xsq[:, None] + Xsq[None, :])
    R2 = np.clip(R2, 0, np.inf)
    R_vals = np.unique(R2.flatten())
    R_vals = R_vals[R_vals > 1e-8]
    
    l_min = np.sqrt(R_vals.min()) / 2.
    l_max = np.sqrt(R_vals.max()) * 2.
    
    return l_min, l_max

## Test 0

In [9]:
## 9 superspots
N = 9000
spatial = []
dependency = []
for i in range(3):
    for j in range(3):
        spatial.append(np.random.multivariate_normal(np.array([j-1,1-i]),np.identity(2)*0.2,1000))
        d = []
        if (i-1) > -1:
            d.append((i-1)*3+j)
        if (j-1) > -1:
            d.append(i*3+j-1)
        dependency.append(d)
        print(j-1,1-i,d)
spatial = np.vstack(spatial)
spatial.shape

-1 1 []
0 1 [0]
1 1 [1]
-1 0 [0]
0 0 [1, 3]
1 0 [2, 4]
-1 -1 [3]
0 -1 [4, 6]
1 -1 [5, 7]


(9000, 2)

In [10]:
l = 0.8
#cov = RBF_kernel(spatial,0.8)
cov = laplacian_kernel(spatial)
mean = np.zeros(N)
Y = np.random.multivariate_normal(mean,cov,200)

In [11]:
multivariate_normal.logpdf(Y, mean, cov)

eps: 8.056818952814166e-07


array([9676.39232825, 9642.95021206, 9810.85496288, 9714.7454243 ,
       9659.15887891, 9502.97832548, 9621.13615196, 9675.84574254,
       9694.71323467, 9699.90048492, 9720.60993432, 9631.66666866,
       9723.20596772, 9558.5700378 , 9662.37409481, 9636.05407331,
       9564.66148917, 9671.29461373, 9657.14092893, 9791.69297996,
       9674.04119967, 9739.67164113, 9787.19939283, 9691.40435195,
       9675.14854213, 9650.02562732, 9662.97819741, 9588.23832363,
       9497.5184583 , 9555.02738034, 9733.68603872, 9531.86828331,
       9565.4621766 , 9604.50702129, 9648.31116133, 9580.47396614,
       9579.39559254, 9591.73914673, 9744.67539496, 9667.37981651,
       9555.43435876, 9641.44916775, 9660.64314063, 9731.19314037,
       9685.26606149, 9663.80112295, 9554.10298934, 9543.69736   ,
       9538.46223033, 9653.94254798, 9636.27820788, 9793.34774272,
       9733.05029184, 9575.81280047, 9639.25682629, 9620.25562001,
       9628.76392424, 9696.07039447, 9642.05565199, 9563.09861

## Test 1

In [7]:
## 9 superspots
N = 9000
spatial = []
dependency = []
for i in range(3):
    for j in range(3):
        spatial.append(np.random.multivariate_normal(np.array([j-1,1-i]),np.identity(2)*0.2,1000))
        d = []
        if (i-1) > -1:
            d.append((i-1)*3+j)
        if (j-1) > -1:
            d.append(i*3+j-1)
        dependency.append(d)
        print(j-1,1-i,d)
spatial = np.vstack(spatial)
spatial.shape

# spatial = np.random.randn(N,2)
# dependency = [[],[0],[0,1],[1,2],[2,3],[3,4],[4,5],[5,6]]
# mean = np.random.randn(10000)
# cut = np.linspace(0,N,9).astype(int)
# ss_loc = [list(range(cut[i],cut[i+1])) for i in range(8)]

-1 1 []
0 1 [0]
1 1 [1]
-1 0 [0]
0 0 [1, 3]
1 0 [2, 4]
-1 -1 [3]
0 -1 [4, 6]
1 -1 [5, 7]


(9000, 2)

In [8]:
cov = laplacian_kernel(spatial)
mean = np.zeros(N)
Y = np.random.multivariate_normal(mean,cov,200)

In [9]:
Y

array([[ 1.29727764,  1.05465461,  1.70272835, ...,  1.40716251,
         0.66534525,  0.49891368],
       [ 1.2276277 ,  1.08637652,  1.30093299, ..., -0.84486675,
        -1.0382536 , -0.86213476],
       [ 1.62990907,  0.87923236,  1.18367949, ...,  0.50586848,
         1.21587636,  0.17273494],
       ...,
       [ 0.59103518, -0.83727368, -0.06494127, ..., -0.74878414,
        -1.1899472 , -0.55435183],
       [-1.70051099, -0.69348697, -1.77794094, ..., -0.45916863,
         0.45698769,  0.20317698],
       [-0.60105843, -1.63014763, -1.13555041, ...,  0.74460248,
         1.02189369,  1.75775647]])

In [10]:
likelihood_1 = multivariate_normal.logpdf(Y, mean, cov)
likelihood_1

eps: 8.019729069114857e-07


array([9677.66288126, 9539.82151642, 9565.86453337, 9692.15329299,
       9667.88173557, 9630.43646993, 9664.55102955, 9741.1403455 ,
       9608.66638227, 9578.75779831, 9730.86758035, 9658.90688015,
       9570.37493995, 9568.56465806, 9772.89946809, 9682.50435345,
       9552.26041247, 9665.72276059, 9585.11589172, 9562.05466387,
       9805.7429111 , 9612.87863031, 9668.57741937, 9644.99950481,
       9609.95441297, 9702.77272293, 9760.01607336, 9604.900401  ,
       9686.86246275, 9702.23477733, 9713.29164202, 9673.20628105,
       9571.39202529, 9677.78626862, 9679.43406429, 9677.96717538,
       9602.38708499, 9657.0466424 , 9704.31425334, 9634.850724  ,
       9718.43748722, 9586.01179273, 9641.64267106, 9609.76640281,
       9636.38540688, 9499.80021955, 9762.75113819, 9642.60293147,
       9596.01362565, 9556.83287893, 9765.76462702, 9582.00224764,
       9734.44155846, 9769.36513767, 9624.72591534, 9728.06430302,
       9655.7558204 , 9645.56146925, 9592.50759143, 9618.89639

In [11]:
ss_loc = [list(range(1000*i, 1000*(i+1))) for i in range(9)]

In [29]:
model_2 = Gaussian_gen(mean,cov,spatial,ss_loc,dependency=dependency,kernel='RBF',l=0.8)

[[1.         0.59894227 0.37245344 0.58798847 0.36358244 0.21886323
  0.35648358 0.22164918 0.13249949]
 [0.59894227 1.         0.60936088 0.37136635 0.59764641 0.36541623
  0.21977752 0.36288944 0.22122248]
 [0.37245344 0.60936088 1.         0.22629613 0.36418234 0.58762573
  0.13392382 0.22113063 0.3557478 ]
 [0.58798847 0.37136635 0.22629613 1.         0.61834961 0.37222367
  0.59180784 0.37696177 0.22534369]
 [0.36358244 0.59764641 0.36418234 0.61834961 1.         0.59906662
  0.36773837 0.60719757 0.36442765]
 [0.21886323 0.36541623 0.58762573 0.37222367 0.59906662 1.
  0.22029979 0.3637518  0.59335703]
 [0.35648358 0.21977752 0.13392382 0.59180784 0.36773837 0.22029979
  1.         0.60563216 0.36845616]
 [0.22164918 0.36288944 0.22113063 0.37696177 0.60719757 0.3637518
  0.60563216 1.         0.59778924]
 [0.13249949 0.22122248 0.3557478  0.22534369 0.36442765 0.59335703
  0.36845616 0.59778924 1.        ]]
[]
det: -4295.117522334454
[-0.00314335]
det: -4335.419685739484
[-0.002

In [19]:
likelihood_2 = model_2.gs_nll_sep(Y.T)
likelihood_2

array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, in

In [16]:
print(likelihood_1-likelihood_2)

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]


In [17]:
model_3 = Gaussian(mean,cov,spatial,ss_loc,dependency=[[]]*9,kernel='RBF',l=0.8)
likelihood_3 = model_3.gs_nll_sep(Y.T)

316
295
310
289
298
289
313
297
285


In [21]:
print(likelihood_3)

[-7598.99229431 -7629.48116858 -7619.97595529 -7550.00217159
 -7612.94454487 -7622.96862256 -7686.75506969 -7598.96112408
 -7592.23490549 -7577.14233449 -7552.11703457 -7552.65651036
 -7624.7758211  -7552.78746133 -7554.86891622 -7630.25920768
 -7643.45759    -7571.56736976 -7637.35085587 -7623.37883261
 -7649.78524757 -7571.86582808 -7649.46715962 -7600.48114096
 -7544.59362964 -7573.38093977 -7646.84343299 -7559.97716664
 -7615.87227734 -7702.34240025 -7590.53013053 -7588.74011536
 -7590.44965499 -7581.45877773 -7598.10285861 -7619.64258939
 -7545.20977793 -7551.80612553 -7643.09574327 -7640.70139836
 -7555.06736936 -7694.13270064 -7599.19838972 -7633.85576053
 -7588.99626569 -7658.52441802 -7572.04249776 -7576.98166125
 -7631.06610103 -7613.59689126 -7649.72452571 -7569.00970384
 -7580.88203611 -7700.37015828 -7635.76492145 -7606.04102779
 -7675.61270271 -7671.66711021 -7578.84875293 -7606.05404597
 -7605.26138174 -7546.66851175 -7604.60025868 -7586.03737671
 -7618.79193516 -7603.56

## Test 2

In [22]:
## 900 superspots
N = 9000
spatial = []
dependency = []
for i in range(30):
    for j in range(30):
        spatial.append(np.random.multivariate_normal(np.array([j-1,1-i]),np.identity(2)*0.8,10))
        d = []
        for i_i in range(i-2,i+1):
            for j_j in range(j-2,j+1):
                if i_i > -1 and j_j > -1:
                    d.append(i_i*30+j_j)
        d.pop()
        print(j-1,1-i,d)
        dependency.append(d)
spatial = np.vstack(spatial)
spatial.shape

-1 1 []
0 1 [0]
1 1 [0, 1]
2 1 [1, 2]
3 1 [2, 3]
4 1 [3, 4]
5 1 [4, 5]
6 1 [5, 6]
7 1 [6, 7]
8 1 [7, 8]
9 1 [8, 9]
10 1 [9, 10]
11 1 [10, 11]
12 1 [11, 12]
13 1 [12, 13]
14 1 [13, 14]
15 1 [14, 15]
16 1 [15, 16]
17 1 [16, 17]
18 1 [17, 18]
19 1 [18, 19]
20 1 [19, 20]
21 1 [20, 21]
22 1 [21, 22]
23 1 [22, 23]
24 1 [23, 24]
25 1 [24, 25]
26 1 [25, 26]
27 1 [26, 27]
28 1 [27, 28]
-1 0 [0]
0 0 [0, 1, 30]
1 0 [0, 1, 2, 30, 31]
2 0 [1, 2, 3, 31, 32]
3 0 [2, 3, 4, 32, 33]
4 0 [3, 4, 5, 33, 34]
5 0 [4, 5, 6, 34, 35]
6 0 [5, 6, 7, 35, 36]
7 0 [6, 7, 8, 36, 37]
8 0 [7, 8, 9, 37, 38]
9 0 [8, 9, 10, 38, 39]
10 0 [9, 10, 11, 39, 40]
11 0 [10, 11, 12, 40, 41]
12 0 [11, 12, 13, 41, 42]
13 0 [12, 13, 14, 42, 43]
14 0 [13, 14, 15, 43, 44]
15 0 [14, 15, 16, 44, 45]
16 0 [15, 16, 17, 45, 46]
17 0 [16, 17, 18, 46, 47]
18 0 [17, 18, 19, 47, 48]
19 0 [18, 19, 20, 48, 49]
20 0 [19, 20, 21, 49, 50]
21 0 [20, 21, 22, 50, 51]
22 0 [21, 22, 23, 51, 52]
23 0 [22, 23, 24, 52, 53]
24 0 [23, 24, 25, 53, 54]
25 0 [24

(9000, 2)

In [23]:
spatial

array([[  0.05358027,   0.35096415],
       [ -2.00468461,   0.41092008],
       [ -0.99038848,   0.66499494],
       ...,
       [ 27.50198769, -28.64120342],
       [ 27.62668085, -28.28952938],
       [ 28.84877734, -28.56193106]])

In [24]:
l_min,l_max = get_l_limits(spatial.T)
space = np.logspace(np.log10(l_min), np.log10(l_max), 10)
space

array([1407.38437333, 1641.75474133, 1915.15458163, 2234.08343474,
       2606.12320346, 3040.11839755, 3546.3863945 , 4136.96271474,
       4825.88714239, 5629.53749331])

In [20]:
l = 0.01
cov = RBF_kernel(spatial,2000)
mean = np.zeros(N)

In [21]:
cov

array([[1.        , 0.99999983, 0.99999943, ..., 0.99981073, 0.99979691,
        0.99980849],
       [0.99999983, 1.        , 0.99999939, ..., 0.99981156, 0.99979714,
        0.99980906],
       [0.99999943, 0.99999939, 1.        , ..., 0.99979029, 0.99977543,
        0.99978779],
       ...,
       [0.99981073, 0.99981156, 0.99979029, ..., 1.        , 0.99999918,
        0.99999989],
       [0.99979691, 0.99979714, 0.99977543, ..., 0.99999918, 1.        ,
        0.99999964],
       [0.99980849, 0.99980906, 0.99978779, ..., 0.99999989, 0.99999964,
        1.        ]])

In [18]:
Y = np.random.multivariate_normal(mean,cov,200)
likelihood_4 = multivariate_normal.logpdf(Y, mean, cov, allow_singular = True)
likelihood_4

array([-12730.39775729, -12629.86353084, -12856.94944374, -12826.45465341,
       -12701.95580482, -12838.06670866, -12786.02161908, -12767.25596832,
       -12686.00826903, -12800.77888523, -12706.92688168, -12654.58972984,
       -12817.53509089, -12766.76758385, -12658.4973617 , -12697.72514758,
       -12788.19559384, -12756.75196618, -12761.02696204, -12705.15462501,
       -12790.66038136, -12835.94011022, -12796.8221687 , -12748.01474654,
       -12785.9504986 , -12661.90766572, -12715.4158938 , -12779.27043072,
       -12753.82783245, -12679.17024229, -12701.53241931, -12720.9413291 ,
       -12776.15650683, -12831.16407091, -12842.87821073, -12778.87288523,
       -12770.67473708, -12797.60493389, -12793.48784439, -12709.70602491,
       -12744.74952638, -12853.00459517, -12755.45577009, -12714.50965405,
       -12866.24223763, -12560.96265729, -12698.18686624, -12729.76406729,
       -12827.71006524, -12787.83551144, -12776.3650922 , -12688.51255542,
       -12720.30418177, -

In [19]:
ss_loc = [list(range(10*i, 10*(i+1))) for i in range(900)]
model_5 = Gaussian(mean,cov,spatial,ss_loc,dependency=dependency,kernel='RBF',l=0.01)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [20]:
likelihood_5 = model_5.gs_nll_sep(Y.T)
print(likelihood_4-likelihood_5)

[ 8.34112953  7.04775877 13.95386109 15.91672855 14.76929411 19.40152929
 17.46886248  9.08099774  7.87386012 14.48274956 16.80378302 14.22803228
  6.2711115  18.44721041 10.62187273 15.7061917  15.28882018 14.29443844
  9.26870808 15.26397126 12.00109952 17.7104031  11.48411331  8.62344162
 13.18235915 14.81156338 14.92842213 15.81408369 15.21933596 17.5119041
 12.22077894 10.81701356 18.0490416  16.3475818  23.06263016  7.62072149
 16.14318027 12.05292038 13.94508413 16.67194628 18.08007399 10.73867536
 15.82101245 16.49958953 22.92288573 21.06495912  7.96816741 11.61896329
 18.40309176 19.43827296 18.29795761 11.32626461 20.08704117 10.42537331
 20.60176909 25.76587513 12.14030826 17.25218817 10.4451668  11.30782237
 20.85838621 13.81863378 13.73304435 14.34766715  9.57808525 17.34432456
 16.90516101 12.0483742  23.78313767 13.08598304  9.71961807 16.71212237
 14.01515707 13.10806254 11.95995474 18.96374162 19.11641895 13.57358539
 19.65238289 20.29889617  6.71910725 24.10889504 20.

In [21]:
model_6 = Gaussian(mean,cov,spatial,ss_loc,dependency=[[]]*900,kernel='RBF',l=0.01)
likelihood_6 = model_6.gs_nll_sep(Y.T)
print(likelihood_5-likelihood_6)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [27]:
cov

array([[1.        , 0.74951767, 0.73792676, ..., 0.        , 0.        ,
        0.        ],
       [0.74951767, 1.        , 0.9094827 , ..., 0.        , 0.        ,
        0.        ],
       [0.73792676, 0.9094827 , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.4159953 ,
        0.18284992],
       [0.        , 0.        , 0.        , ..., 0.4159953 , 1.        ,
        0.08334105],
       [0.        , 0.        , 0.        , ..., 0.18284992, 0.08334105,
        1.        ]])